# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [4]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [5]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [6]:
import os
%env HF_HOME=/root/autodl-tmp/huggingface/cache
%env HF_ENDPOINT = https://hf-mirror.com
os.environ['HF_HOME'] = '/root/autodl-tmp/huggingface/cache'
os.environ['HF_HUB_CACHE'] = '/root/autodl-tmp/huggingface/cache'
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

env: HF_HOME=/root/autodl-tmp/huggingface/cache
env: HF_ENDPOINT=https://hf-mirror.com


In [7]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])

# 加载量化后模型(与微调的 revision 保持一致）
base_model = AutoModel.from_pretrained(model_name_or_path,
                                      quantization_config=q_config,
                                      device_map='auto',
                                      trust_remote_code=True,
                                      revision='b098244')

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [15]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [16]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path,
                                          trust_remote_code=True,
                                          revision='b098244')

## 使用原始 ChatGLM3-6B 模型

In [17]:
input_text = "解释下乾卦是什么？"

In [18]:
response, history = base_model.chat(tokenizer, query=input_text)

In [19]:
print(response)

乾卦是八卦之一，也是八宫图之一，它是由两个阴爻夹一个阳爻构成，象征着天、云和雷。乾卦的意义非常广泛，它不仅代表天、云和雷，也代表着 strength, leadership, and authority. It is often associated with the physical and material world, as well as the natural world such as the sky and clouds. It is also said to represent the ultimate authority and power, as well as the process of change and transformation.


#### 询问一个64卦相关问题（应该不在 ChatGLM3-6B 预训练数据中）

In [20]:
response, history = base_model.chat(tokenizer, query="周易中的讼卦是什么？", history=history)
print(response)

讼卦是八卦之一，也是八宫图之一。它由两个阳爻夹一个阴爻构成，象征着天、云和雷。讼卦的意义非常广泛，它不仅代表天、云和雷，也代表着争端、诉讼、诉讼程序和法律。它也象征着自然现象，如风和雨。在这个卦象中，阳爻代表刚强和坚定，阴爻则代表柔弱和温和。这个卦象主要表达的是在争端中，柔弱胜过刚强的道理。它建议我们在处理争端和诉讼时要保持柔和的态度，以柔克刚，才能取得胜利。


## 使用微调后的 ChatGLM3-6B

### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed)) - 请根据训练时间戳修改 timestamp 

In [21]:
from peft import PeftModel, PeftConfig

epochs = 3
# timestamp = "20240118_164514"
timestamp = "20240724_153657"

peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag=f"ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-{timestamp}"

In [30]:
def compare_chatglm_results(query, base_model, qlora_model, training_tag):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    # ft_out = qlora_model.generate(**inputs, max_new_tokens=512)
    # ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    ft_response = qlora_model.chat(tokenizer,query)
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\n微调后（{training_tag}）：\n{ft_response}")
    return base_response, ft_response

### 微调前后效果对比

In [31]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model, training_tag)

问题：解释下乾卦是什么？

原始输出：
在周易中，乾卦是六十四卦之首，象征天、龙，以及纯粹的阳和健。其卦辞为“元、亨、利、贞”，意味着大吉大利，吉利的贞卜。乾卦的核心哲学是天道刚健，君子观此卦象，从而以天为法，自强不息。乾卦所代表的刚健、正直、公允的实质，以及自强不息的精神，是在正确的道路上永远亨通的象征。在事业、经商、求名、婚恋等方面，都具有兴盛强健、名利双收的象征，但也需要注意骄傲自满的情绪，保持警惕和谨慎，方可充分发挥才智，保证成功。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
('在周易中，乾卦是六十四卦之首，象征天、龙，以及纯粹的阳和健。其卦辞为“元、亨、利、贞”，意味着大吉大利，吉利的贞卜。乾卦的核心哲学是天道刚健，君子观此卦象，从而以天为法，自强不息。乾卦所代表的刚健、正直、公允的实质，以及自强不息的精神，是在正确的道路上永远亨通的象征。在事业、经商、求名、婚恋等方面，都具有兴盛强健、名利双收的象征，但也需要注意骄傲自满的情绪，保持警惕和谨慎，方可充分发挥才智，保证成功。', [{'role': 'user', 'content': '解释下乾卦是什么？'}, {'role': 'assistant', 'metadata': '', 'content': '在周易中，乾卦是六十四卦之首，象征天、龙，以及纯粹的阳和健。其卦辞为“元、亨、利、贞”，意味着大吉大利，吉利的贞卜。乾卦的核心哲学是天道刚健，君子观此卦象，从而以天为法，自强不息。乾卦所代表的刚健、正直、公允的实质，以及自强不息的精神，是在正确的道路上永远亨通的象征。在事业、经商、求名、婚恋等方面，都具有兴盛强健、名利双收的象征，但也需要注意骄傲自满的情绪，保持警惕和谨慎，方可充分发挥才智，保证成功。'}])


In [24]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model, training_tag)

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，乾象征刚健，坎象征险陷。这两者相背而行，预示着将会有争讼之事。占卦得此卦，虽然开始时有利可图，但最终会陷入凶险之中。

讼卦象征着事与愿违，凡事不顺，可能会遭遇小人加害，需要时刻防范陷阱。因此，谋事之初务必慎之又慎，不得固执已见，极力避免介入诉讼纠纷的争执之中。不如退而让人，求得化解，安于正理，可免除意外之灾。对于事业、经商、求名、婚恋等方面都有着相应的警示和建议。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240724_153657）：
[gMASK]sop 周易中的讼卦是什么? 在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，乾象征刚健，坎象征险陷。这两者相背而行，预示着将会有争讼之事。占卦得此卦，虽然开始时有利可图，但最终会陷入凶险之中。

讼卦象征着事与愿违，凡事不顺，可能会遭遇小人加害，需要时刻防范陷阱。因此，谋事之初务必慎之又慎，不得固执已见，极力避免介入诉讼纠纷的争执之中。不如退而让人，求得化解，安于正理，可免除意外之灾。对于事业、经商、求名、婚恋等方面都有着相应的警示和建议。


In [25]:
base_response, ft_response = compare_chatglm_results("师卦是什么？", base_model, qlora_model, training_tag)

问题：师卦是什么？

原始输出：
师卦是一个由两个异卦相叠而成的卦象，下卦为坎（水），上卦为坤（地）。这一卦象代表着军队和指挥军事的总体情况，预示着吉祥和无灾祸。师卦的核心哲学是兵众之情，取法于地中有水，象征着容纳众人和畜养大众的含义。在周易哲学中，只有德高望重的长者来统率军队，才能带来吉祥和无咎。师卦预示着时运的包容与修行待时，财运的珍惜与财库，家宅的联姻和喜庆，以及身体的调气无忧。在事业上，师卦预示着困难重重，需要遵守正规，与他人合作，谨慎行事，果断而不冒险，始终保持机动灵活。经商则需要坚强的精神和高尚的商业道德，加强沟通，方可摆脱困境。在求名与婚恋中，均需要专注慎重。总的来说，师卦预示着克服困难与坚定追求的人生态度。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset(fixed))-20240724_153657）：
[gMASK]sop 师卦是什么？ 师卦是一个由两个异卦相叠而成的卦象，下卦为坎（水），上卦为坤（地）。这一卦象代表着军队和指挥军事的总体情况，预示着吉祥和无灾祸。师卦的核心哲学是兵众之情，取法于地中有水，象征着容纳众人和畜养大众的含义。在周易哲学中，只有德高望重的长者来统率军队，才能带来吉祥和无咎。师卦预示着时运的包容与修行待时，财运的珍惜与财库，家宅的联姻和喜庆，以及身体的调气无忧。在事业上，师卦预示着困难重重，需要遵守正规，与他人合作，谨慎行事，果断而不冒险，始终保持机动灵活。经商则需要坚强的精神和高尚的商业道德，加强沟通，方可摆脱困境。在求名与婚恋中，均需要专注慎重。总的来说，师卦预示着克服困难与坚定追求的人生态度。


## 其他模型（错误数据或训练参数）

#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [26]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e3 = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=3, automade-dataset)"

In [27]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e3, training_tag)

问题：解释下乾卦是什么？

原始输出：
在周易中，乾卦是六十四卦之首，象征天、龙，以及纯粹的阳和健。其卦辞为“元、亨、利、贞”，意味着大吉大利，吉利的贞卜。乾卦的核心哲学是天道刚健，君子观此卦象，从而以天为法，自强不息。乾卦所代表的刚健、正直、公允的实质，以及自强不息的精神，是在正确的道路上永远亨通的象征。在事业、经商、求名、婚恋等方面，都具有兴盛强健、名利双收的象征，但也需要注意骄傲自满的情绪，保持警惕和谨慎，方可充分发挥才智，保证成功。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，象征天、龙，以及纯粹的阳和健。其卦辞为“元、亨、利、贞”，意味着大吉大利，吉利的贞卜。乾卦的核心哲学是天道刚健，君子观此卦象，从而以天为法，自强不息。乾卦所代表的刚健、正直、公允的实质，以及自强不息的精神，是在正确的道路上永远亨通的象征。在事业、经商、求名、婚恋等方面，都具有兴盛强健、名利双收的象征，但也需要注意骄傲自满的情绪，保持警惕和谨慎，方可充分发挥才智，保证成功。


In [28]:
base_response, ft_response = compare_chatglm_results("地水师卦是什么？", base_model, qlora_model_e3, training_tag)

问题：地水师卦是什么？

原始输出：
地水师卦是一个由两个卦相交而得到的卦象，下卦为坎（水），上卦为坤（地）。这一卦象代表着地元素的特性，象征着地气蒸腾，以及运势的顺境。地水师卦预示着势力强大，财运库满，意味着顺应自然，顺应形势，可以安心待时，财运良好。然而，地水师卦 also预示着潜在的危险和风险，建议谨慎对待，以免损失。在事业上，应谨慎处理，不要过于依赖他人的帮助，以免陷入口水战的泥潭。在经商方面，需要谨慎选择商业机会，避免冲动行事，否则将会有损财。在婚恋方面，需要谨慎选择伴侣，避免过于冲动的爱情，才能确保婚姻美满。总体而言，地水师卦提示着顺应自然，谨慎行事，谨慎冒险，才能获得成功。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 地水师卦是什么？ 地水师卦是一个由两个异卦相叠而成的卦象，下卦为坎（水），上卦为坤（地）。这一卦象代表着地元素的能量，预示着柔静的力量即将消逝，柔弱胜过刚强，预示着事物将朝著相反的方向发展。地水师卦预示着柔静的特性即将消失，柔弱将战胜刚强，预示着事物将朝著相反的方向发展。在哲学和心理学领域，地水师卦预示着柔性的力量即将消失，强性的力量即将产生。在生活和事业中，地水师卦预示着柔性的力量即将消失，强性的力量将有助于成功。在情感方面，地水师卦预示着爱情我将走运，婚姻我将顺利，代表着幸福和成功。在商业领域，地水师卦预示着成功和收获，代表着成功和财富。地水师卦预示着柔性的能量即将消失，强性的能量即将产生，柔性的力量将消失，强性的力量将产生，预示着事物将朝著相反的方向发展。


In [29]:
base_response, ft_response = compare_chatglm_results("周易中的讼卦是什么", base_model, qlora_model_e3, training_tag)

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，乾象征刚健，坎象征险陷。这两者相背而行，预示着将会有争讼之事。占卦得此卦，虽然开始时有利可图，但最终会陷入凶险之中。

讼卦象征着事与愿违，凡事不顺，可能会遭遇小人加害，需要时刻防范陷阱。因此，谋事之初务必慎之又慎，不得固执已见，极力避免介入诉讼纠纷的争执之中。不如退而让人，求得化解，安于正理，可免除意外之灾。对于事业、经商、求名、婚恋等方面都有着相应的警示和建议。


微调后（ChatGLM3-6B(Epoch=3, automade-dataset)）：
[gMASK]sop 周易中的讼卦是什么? 在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，乾象征刚健，坎象征险陷。这两者相背而行，预示着将会有争讼之事。占卦得此卦，虽然开始时有利可图，但最终会陷入凶险之中。

讼卦象征着事与愿违，凡事不顺，可能会遭遇小人加害，需要时刻防范陷阱。因此，谋事之初务必慎之又慎，不得固执已见，极力避免介入诉讼纠纷的争执之中。不如退而让人，求得化解，安于正理，可免除意外之灾。对于事业、经商、求名、婚恋等方面都有着相应的警示和建议。


#### 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [32]:
from peft import PeftModel, PeftConfig

epochs = 50
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
qlora_model_e50_handmade = PeftModel.from_pretrained(base_model, peft_model_path)
training_tag = f"ChatGLM3-6B(Epoch=50, handmade-dataset)"

In [33]:
base_response, ft_response = compare_chatglm_results("解释下乾卦是什么？", base_model, qlora_model_e50_handmade, training_tag)

问题：解释下乾卦是什么？

原始输出：
在周易中，乾卦是六十四卦之首，象征天、龙，以及纯粹的阳和健。其卦辞为“元、亨、利、贞”，意味着大吉大利，吉利的贞卜。乾卦的核心哲学是天道刚健，君子观此卦象，从而以天为法，自强不息。乾卦所代表的刚健、正直、公允的实质，以及自强不息的精神，是在正确的道路上永远亨通的象征。在事业、经商、求名、婚恋等方面，都具有兴盛强健、名利双收的象征，但也需要注意骄傲自满的情绪，保持警惕和谨慎，方可充分发挥才智，保证成功。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
('在周易中，乾卦是六十四卦之首，象征天、龙，以及纯粹的阳和健。其卦辞为“元、亨、利、贞”，意味着大吉大利，吉利的贞卜。乾卦的核心哲学是天道刚健，君子观此卦象，从而以天为法，自强不息。乾卦所代表的刚健、正直、公允的实质，以及自强不息的精神，是在正确的道路上永远亨通的象征。在事业、经商、求名、婚恋等方面，都具有兴盛强健、名利双收的象征，但也需要注意骄傲自满的情绪，保持警惕和谨慎，方可充分发挥才智，保证成功。', [{'role': 'user', 'content': '解释下乾卦是什么？'}, {'role': 'assistant', 'metadata': '', 'content': '在周易中，乾卦是六十四卦之首，象征天、龙，以及纯粹的阳和健。其卦辞为“元、亨、利、贞”，意味着大吉大利，吉利的贞卜。乾卦的核心哲学是天道刚健，君子观此卦象，从而以天为法，自强不息。乾卦所代表的刚健、正直、公允的实质，以及自强不息的精神，是在正确的道路上永远亨通的象征。在事业、经商、求名、婚恋等方面，都具有兴盛强健、名利双收的象征，但也需要注意骄傲自满的情绪，保持警惕和谨慎，方可充分发挥才智，保证成功。'}])


In [34]:
base_response, ft_response = compare_chatglm_results("地水师卦", base_model, qlora_model_e50_handmade, training_tag)

问题：地水师卦

原始输出：
卦象为地雷师卦,由下卦坎(水)和上卦坎(地)组成,卦象中下卦为坎,上卦为坎,代表着地雷,预示着云霓密布,大事难成,自我投资平实,建议观望,避免决策,等待时机,总体建议是静观等待, hints@gmail.com。地雷师卦的核心卦象是地雷,象征着云和雨,预示着云霓密布,但大事难以成,需要等待时机。在投资方面,需要观望,避免决策,应该静待时机,不要强求。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
('卦象为地雷师,由下卦坎(水)和上卦坤(地)组成。这一卦象代表军队,预示着吉祥和无灾祸。在战争实践中,地雷师卦表示军队已经收到有益的讯息,预示着胜利和幸福。在数学上,地雷师卦预示着会赚取利润,但也会 safe and secure财富。但地雷师卦并不是适用于 every situation,它只适用于专门从事军事活动的人,卦象预示着只会有利润,而不适用于其他活动。地雷师卦的核心信息是,只有掌握地雷之力的军队,才能获得成功。', [{'role': 'user', 'content': '地水师卦'}, {'role': 'assistant', 'metadata': '', 'content': '卦象为地雷师,由下卦坎(水)和上卦坤(地)组成。这一卦象代表军队,预示着吉祥和无灾祸。在战争实践中,地雷师卦表示军队已经收到有益的讯息,预示着胜利和幸福。在数学上,地雷师卦预示着会赚取利润,但也会 safe and secure财富。但地雷师卦并不是适用于 every situation,它只适用于专门从事军事活动的人,卦象预示着只会有利润,而不适用于其他活动。地雷师卦的核心信息是,只有掌握地雷之力的军队,才能获得成功。'}])


In [35]:
base_response, ft_response = compare_chatglm_results("天水讼卦", base_model, qlora_model_e50_handmade, training_tag)

问题：天水讼卦

原始输出：
天水讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，乾象征刚健，坎象征险陷。这两者相背而行，预示着将会有争讼之事。占卦得此卦，虽然开始时有利可图，但最终会陷入凶险之中。

在这个卦象中，glory和 calamity都指向着凶险的方向，因此建议你避免涉及法律诉讼的纠纷，以及过度追求成功。同时，也要谨慎做决策，避免冒险行为，避免将资金投入险境之中。


微调后（ChatGLM3-6B(Epoch=50, handmade-dataset)）：
('在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，乾象征刚健，坎象征险陷。这两者相背而行，预示着将会有争讼之事。占卦得此卦，虽然开始时有利可图，但最终会陷入凶险之中。\n\n讼卦象征着事与愿违，凡事不顺，可能会遭遇小人加害，需要时刻防范陷阱。因此，谋事之初务必慎之又慎，不得固执已见，极力避免介入诉讼纠纷的争执之中。不如退而让人，求得化解，安于正理，可免除意外之灾。对于事业、经商、求名、婚恋等方面都有着相应的警示和建议。', [{'role': 'user', 'content': '天水讼卦'}, {'role': 'assistant', 'metadata': '', 'content': '在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，乾象征刚健，坎象征险陷。这两者相背而行，预示着将会有争讼之事。占卦得此卦，虽然开始时有利可图，但最终会陷入凶险之中。\n\n讼卦象征着事与愿违，凡事不顺，可能会遭遇小人加害，需要时刻防范陷阱。因此，谋事之初务必慎之又慎，不得固执已见，极力避免介入诉讼纠纷的争执之中。不如退而让人，求得化解，安于正理，可免除意外之灾。对于事业、经商、求名、婚恋等方面都有着相应的警示和建议。'}])
